In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [5]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-EMERGWATER/DRM-EMERGWATER.csv", encoding="cp949")  # cp949로 불러오는 게 안전
print("원본 크기:", df.shape)
display(df.head())

# 2. 자치구 정리
# df["gu"] = df["자치구"].astype(str).str.replace("구$", "", regex=True)

# 3. 수치형 변환
df["양수량(톤/일)"] = pd.to_numeric(df["양수량(톤/일)"], errors="coerce").fillna(0)
df["자가발전기출력(kw)"] = pd.to_numeric(df["자가발전기출력(kw)"], errors="coerce").fillna(0)

# 4. 집계
agg = df.groupby("자치구").agg(
    emergwater_count=("관리번호", "count"),                 # 시설 개수
    emergwater_total_supply=("양수량(톤/일)", "sum"),      # 총 양수량
    emergwater_avg_supply=("양수량(톤/일)", "mean"),      # 평균 양수량
    emergwater_gen_ratio=("자가발전기출력(kw)", lambda x: (x>0).sum()/len(x)) # 자가발전기 비율
).reset_index()

display(agg.head())

# 5. 점수화 (많을수록 안전 → 정방향)
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["emergwater_count","emergwater_total_supply","emergwater_avg_supply","emergwater_gen_ratio"]])

score_df = agg[["자치구"]].copy()
for i, col in enumerate(["emergwater_count","emergwater_total_supply","emergwater_avg_supply","emergwater_gen_ratio"]):
    score_df[col+"_score"] = scaled[:, i]

# 종합 점수 (평균)
score_df["emergwater_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 6. 저장
score_df.to_csv("../../result/DRM/DRM-EMERGWATER_result.csv", index=False, encoding="utf-8-sig")

score_df.head()


원본 크기: (20440, 23)


,관리번호,등록구분,등록년도,모터펌프종류,모터펌프출력(kw),비고,설치년도,설치심도(m),시설구분,시설명(설치장소),...,용도구분,자가발전기종류,자가발전기출력(kw),자치구,전화번호,최종수정일시,최초등록일시,해지날짜,해지사유,해지여부
0,2018_1_0359,1,2018,수중펌프,30,NaN,2002.0,700.0,빌딩,금강산보석사우나,...,생활용수,NaN,NaN,노원구,NaN,2019-04-11 18:07:58.0,2019-04-11 18:07:58.0,NaN,NaN,지정
1,2018_1_0360,1,2018,수중펌프,2.25,NaN,2002.0,150.0,병원,을지병원(별관),...,생활용수,디젤엔진,600,노원구,NaN,2019-04-11 18:07:58.0,2019-04-11 18:07:58.0,NaN,NaN,지정
2,2018_1_0361,1,2018,수중펌프,5.63,NaN,2007.0,250.0,빌딩,궁전사우나,...,생활용수,NaN,NaN,노원구,NaN,2019-04-11 18:07:58.0,2019-04-11 18:07:58.0,NaN,NaN,지정
3,2018_1_0362,1,2018,수중펌프,0.75,NaN,1993.0,20.0,음식점,오장동함흥냉면,...,생활용수,NaN,NaN,노원구,NaN,2019-04-11 18:07:58.0,2019-04-11 18:07:58.0,NaN,NaN,지정
4,2018_1_0363,1,2018,수중펌프,1.5,NaN,1997.0,150.0,빌딩,노원장,...,생활용수,NaN,NaN,노원구,NaN,2019-04-11 18:07:58.0,2019-04-11 18:07:58.0,NaN,NaN,지정


,자치구,emergwater_count,emergwater_total_supply,emergwater_avg_supply,emergwater_gen_ratio
0,강남구,1005,132533.200,131.873831,0.592040
1,강동구,901,130942.000,145.329634,0.150943
2,강북구,503,65008.000,129.240557,0.178926
3,강서구,1118,126821.908,113.436411,0.110912
4,관악구,998,86597.000,86.770541,0.324649


,자치구,emergwater_count_score,emergwater_total_supply_score,emergwater_avg_supply_score,emergwater_gen_ratio_score,emergwater_mean_score
0,강남구,54.956170,38.617354,6.460947,70.529074,42.640886
1,강동구,47.943358,37.787036,7.920172,14.412406,27.015743
2,강북구,21.105866,3.381429,6.175379,17.972433,12.158777
3,강서구,62.575860,35.637094,4.461486,9.319621,27.998515
4,관악구,54.484154,14.646977,1.569685,36.511422,26.803059


gu (자치구명)

emergwater_count_score (비상급수 시설 개수 점수)

emergwater_total_supply_score (총 양수량 점수)

emergwater_avg_supply_score (평균 양수량 점수)

emergwater_gen_ratio_score (자가발전기 비율 점수)

emergwater_mean_score (평균 종합 점수)